In [ ]:
# Import JSON package
import json

# File path
ecoinvent_logins_json_file = 'ecoinvent-logins.json'

# Load the JSON file content as a Python dict object
with open(ecoinvent_logins_json_file) as ecoinvent_logins_json_file:
    ecoinvent_logins = json.load(ecoinvent_logins_json_file)

# Getting started with Brightway

## Initialization

In [1]:
# Brightway package for data management
import bw2data

10:59:40+0100 [warning  ] Can't import `SimaProBlockCSVImporter` - please install `bw2io` with `pip install bw2io[multifunctional]` or install `multifunctional` and `bw_simapro_csv` manually.


### LCA project

> A Brightway project is an isolated set of data sufficient to do LCA calculations.
> 
> Source: https://docs.brightway.dev/en/latest/content/overview/glossary.html#term-Project

In [2]:
# List available project, in alphabetic order
sorted(bw2data.projects)

[Project: 202507-rebecca-brightway-lesson,
 Project: 20251003-learn-brightway-copernicus,
 Project: 20251013-2g-ethanol,
 Project: 20251031-bw2calc-issue-biosphere-flows,
 Project: 3.9.1,
 Project: Brightway4dummies,
 Project: default,
 Project: EXIOBASE 2022,
 Project: template-ecoinvent-3.11-cutoff]

In [ ]:
# Select (or create if not yet available) the project
bw2data.projects.set_current(name=ecoinvent_logins['project_name'])

In [4]:
# Verify the current project name
bw2data.projects.current

'20251120-copernicus-tutorial-getting-started-with-brightway'

### Databases

> A Brightway database is a **container for inventory** nodes. Databases are primarily organization tools, and their precise boundaries are determined by practitioners. Nodes must be associated with one and only one database, but edges can and often do cross from one database to another.
> 
> Brightway does not make a distinction between foreground and background databases.
> 
> Source: https://docs.brightway.dev/en/latest/content/overview/glossary.html#term-Database

In [5]:
# List databases available in the project
bw2data.databases

Databases dictionary with 0 objects

#### Import Ecoinvent database

In [6]:
# Brightway package for importing and exporting data
import bw2io

##### Get Ecoinvent account login details from a different file

You do not want to share it publicly

In [9]:
bw2io.import_ecoinvent_release(
    version='3.11',
    system_model='cutoff',
    username=ecoinvent_logins['username'], # Load from the JSON file
    password=ecoinvent_logins['password'] # Load from the JSON file
)

Applying strategy: normalize_units
Applying strategy: drop_unspecified_subcategories
Applying strategy: ensure_categories_are_tuples
Applied 3 strategies in 0.01 seconds
Graph statistics for `ecoinvent-3.11-biosphere` importer:
9795 graph nodes:
	emission: 9428
	natural resource: 347
	inventory indicator: 15
	economic: 5
0 graph edges:
0 edges to the following databases:
0 unique unlinked edges (0 total):




100%|██████████| 9795/9795 [00:00<00:00, 49778.88it/s]


11:06:10+0100 [info     ] Vacuuming database            
Created database: ecoinvent-3.11-biosphere
Extracting XML data from 25412 datasets
11:06:35+0100 [info     ] Extracted 25412 datasets in 23.24 seconds
Applying strategy: normalize_units
Applying strategy: update_ecoinvent_locations
Applying strategy: remove_zero_amount_coproducts
Applying strategy: remove_zero_amount_inputs_with_no_activity
Applying strategy: remove_unnamed_parameters
Applying strategy: es2_assign_only_product_with_amount_as_reference_product
Applying strategy: assign_single_product_as_activity
Applying strategy: create_composite_code
Applying strategy: drop_unspecified_subcategories
Applying strategy: fix_ecoinvent_flows_pre35
Applying strategy: drop_temporary_outdated_biosphere_flows


/home/nicolas/miniconda3/envs/20251003-learn-brightway-copernicus/lib/python3.12/site-packages/bw2io/strategies/ecospold2.py:1066: UserWarning: Skipping migration 'fix-ecoinvent-flows-pre-35' because it isn't installed
  warnings.warn(


Applying strategy: link_biosphere_by_flow_uuid
Applying strategy: link_internal_technosphere_by_composite_code
Applying strategy: delete_exchanges_missing_activity
Applying strategy: delete_ghost_exchanges
Applying strategy: remove_uncertainty_from_negative_loss_exchanges
Applying strategy: fix_unreasonably_high_lognormal_uncertainties
Applying strategy: convert_activity_parameters_to_list
Applying strategy: add_cpc_classification_from_single_reference_product
Applying strategy: delete_none_synonyms
Applying strategy: update_social_flows_in_older_consequential
Applying strategy: set_lognormal_loc_value
Applied 22 strategies in 7.42 seconds
Graph statistics for `ecoinvent-3.11-cutoff` importer:
25412 graph nodes:
	process: 25412
855034 graph edges:
	biosphere: 541321
	technosphere: 288301
	production: 25412
855034 edges to the following databases:
	ecoinvent-3.11-biosphere: 541321
	ecoinvent-3.11-cutoff: 313713
0 unique unlinked edges (0 total):


11:06:44+0100 [warning  ] Not able to d

100%|██████████| 25412/25412 [00:27<00:00, 916.19it/s] 


11:07:12+0100 [info     ] Vacuuming database            
Created database: ecoinvent-3.11-cutoff


In [10]:
# Make EcoInvent database usable via a Python object
ecoinvent = bw2data.Database('ecoinvent-3.11-cutoff')

## Modelling the LCA of a simple Ecoinvent activity

### Selecting an activity

Imagine that you want to calculate the GHG footprint of 1 kWh of low voltage electricity in the Netherlands.

We can look on Ecoinvent's ecoQuery if such activity exists and how it is called: https://ecoquery.ecoinvent.org/

We will use this one: https://ecoquery.ecoinvent.org/3.11/cutoff/dataset/7105/documentation

In [11]:
# Searching an activity in the database. Here it is Ecoinvent, but it could be any other database in the current project
activities = ecoinvent.search("market for electricity, low voltage")
# There are many activities with this name
activities

['electricity voltage transformation from medium to low voltage, renewable energy products' (kilowatt hour, CH, None),
 'market group for electricity, low voltage' (kilowatt hour, RER, None),
 'market group for electricity, low voltage' (kilowatt hour, CA, None),
 'market group for electricity, low voltage' (kilowatt hour, RAS, None),
 'market group for electricity, low voltage' (kilowatt hour, IN, None),
 'market group for electricity, low voltage' (kilowatt hour, BR, None),
 'market group for electricity, low voltage' (kilowatt hour, CN, None),
 'market group for electricity, low voltage' (kilowatt hour, RAF, None),
 'market group for electricity, low voltage' (kilowatt hour, CN-SGCC, None),
 'market group for electricity, low voltage' (kilowatt hour, RNA, None),
 'market group for electricity, low voltage' (kilowatt hour, RME, None),
 'market group for electricity, low voltage' (kilowatt hour, US, None),
 'market group for electricity, low voltage' (kilowatt hour, GLO, None),
 'mark

In [13]:
# Let's refine it with the country code of the Netherlands 'NL'
activities = ecoinvent.search("market for electricity, low voltage, NL")
# There are still some activities with this name
activities

['electricity production, photovoltaic, 3kWp slanted-roof installation, single-Si, panel, mounted' (kilowatt hour, NL, None),
 'electricity production, photovoltaic, 3kWp slanted-roof installation, multi-Si, panel, mounted' (kilowatt hour, NL, None),
 'electricity production, photovoltaic, 570kWp open ground installation, multi-Si' (kilowatt hour, NL, None),
 'market for electricity, low voltage' (kilowatt hour, NL, None),
 'electricity, low voltage, residual mix' (kilowatt hour, NL, None),
 'market for electricity, medium voltage' (kilowatt hour, NL, None),
 'market for electricity, for reuse in municipal waste incineration only' (kilowatt hour, NL, None),
 'electricity production, wind, 1-3MW turbine, offshore' (kilowatt hour, NL, None),
 'market for electricity, high voltage' (kilowatt hour, NL, None),
 'electricity production, wind, 1-3MW turbine, onshore' (kilowatt hour, NL, None),
 'electricity production, nuclear, pressure water reactor' (kilowatt hour, NL, None),
 'electricity 

In [15]:
# Selecting the right one
activity_low_voltage_electricity_NL = activities[3]
activity_low_voltage_electricity_NL

'market for electricity, low voltage' (kilowatt hour, NL, None)

### Selecting the LCIA indicator

#### List available LCIA methods and indicators

In [20]:
# List all available methods in the current project
lcia_methods = bw2data.methods
list(lcia_methods)

[('ecoinvent-3.11',
  'CML v4.8 2016 no LT',
  'acidification no LT',
  'acidification (incl. fate, average Europe total, A&B) no LT'),
 ('ecoinvent-3.11',
  'CML v4.8 2016 no LT',
  'climate change no LT',
  'global warming potential (GWP100) no LT'),
 ('ecoinvent-3.11',
  'CML v4.8 2016 no LT',
  'ecotoxicity: freshwater no LT',
  'freshwater aquatic ecotoxicity (FAETP inf) no LT'),
 ('ecoinvent-3.11',
  'CML v4.8 2016 no LT',
  'ecotoxicity: marine no LT',
  'marine aquatic ecotoxicity (MAETP inf) no LT'),
 ('ecoinvent-3.11',
  'CML v4.8 2016 no LT',
  'ecotoxicity: terrestrial no LT',
  'terrestrial ecotoxicity (TETP inf) no LT'),
 ('ecoinvent-3.11',
  'CML v4.8 2016 no LT',
  'energy resources: non-renewable no LT',
  'abiotic depletion potential (ADP): fossil fuels no LT'),
 ('ecoinvent-3.11',
  'CML v4.8 2016 no LT',
  'eutrophication no LT',
  'eutrophication (fate not incl.) no LT'),
 ('ecoinvent-3.11',
  'CML v4.8 2016 no LT',
  'human toxicity no LT',
  'human toxicity (HTP 

#### Selecting one LCIA indicator

In [24]:
# Select climate change indicators from EF 3.1 LCIA method that is provided by Ecoinvent
ef3_1_climate_change_lcia_indicator = [m for m in lcia_methods if m[0] == "ecoinvent-3.11" and "climate change" == m[2] and "EF v3.1" == m[1]][0]
ef3_1_climate_change_lcia_indicator

('ecoinvent-3.11',
 'EF v3.1',
 'climate change',
 'global warming potential (GWP100)')

### Setting up the LCA

In [25]:
# Brightway package to compute LCAs or solve LCIs
import bw2calc

In [26]:
# TODO: explain
functional_unit, data_objs, _ = bw2data.prepare_lca_inputs(
    {activity_low_voltage_electricity_NL: 1},
    method=ef3_1_climate_change_lcia_indicator,
)

# Build the LCA
lca = bw2calc.LCA(demand=functional_unit, data_objs=data_objs)

### Running the LCA

In [27]:
# Solve the LCI, necessary to compute the LCA.
# TODO: explain or validate: It calculates the sum of each of the elementary flows due to the functional unit, across the different databases (here only Ecoinvent) 
lca.lci()

In [28]:
# Calculate the LCIA results based on the LCI and the characterization factors from the LCIA method
lca.lcia()

#### Final result

In [ ]:
# Output our final result
lca.score

0.3970655748220028

In [ ]:
# With the right unit
lca.score

0.3970655748220028

In [36]:
type(bw2data.Method(ef3_1_climate_change_lcia_indicator))
bw2data.Method(ef3_1_climate_change_lcia_indicator)

Brightway2 Method: ecoinvent-3.11: EF v3.1: climate change: global warming potential (GWP100)